# 📊 Proyecto: Preparación de Datos con Python

Este notebook desarrolla el proyecto completo de **Preparación de Datos**, utilizando **NumPy y Pandas**, organizado por lecciones.


## step 1 — Generación de datos ficticios con NumPy
Creación de datasets de clientes y transacciones, operaciones básicas y guardado en `.npy`.

In [5]:
# =========================
# Lección 1 - NumPy
# Generación de datos ficticios + métricas básicas + guardado .npy
# =========================

import numpy as np

# 1) Configuración: semilla (reproducibilidad) y tamaños del dataset
np.random.seed(42)
n_customers = 500
n_transactions = 5000

# -------------------------
# 2) Dataset de clientes
# -------------------------
# IDs únicos consecutivos para clientes
customer_ids = np.arange(10000, 10000 + n_customers)

# Variables ficticias de clientes
ages = np.random.randint(18, 71, size=n_customers)          # edades 18-70
tenure_months = np.random.randint(0, 61, size=n_customers)  # antigüedad 0-60 meses

# Tabla NumPy (n_customers x 3): [customer_id, age, tenure_months]
customers = np.column_stack([customer_ids, ages, tenure_months])

# -------------------------
# 3) Dataset de transacciones
# -------------------------
# IDs únicos consecutivos para transacciones
transaction_ids = np.arange(1, n_transactions + 1)

# Asignar un cliente aleatorio a cada transacción (un cliente puede repetir)
tx_customer_id = np.random.choice(customer_ids, size=n_transactions, replace=True)

# Cantidad de productos por transacción (1 a 12)
quantity = np.random.randint(1, 13, size=n_transactions)

# Precio unitario en CLP: gamma (valores positivos, cola a la derecha) + límites realistas
unit_price = np.random.gamma(shape=2.0, scale=10000.0, size=n_transactions)
unit_price = np.clip(unit_price, 500, 150000).astype(int)   # CLP sin decimales

# Total por transacción (CLP)
total = quantity * unit_price

# Tabla NumPy (n_transactions x 5):
# [transaction_id, customer_id, quantity, unit_price, total]
transactions = np.column_stack([transaction_ids, tx_customer_id, quantity, unit_price, total])

# -------------------------
# 4) Operaciones matemáticas básicas (resumen)
# -------------------------
total_sales = total.sum()
avg_ticket = total.mean()
max_ticket = total.max()
min_ticket = total.min()
count_transactions = total.size

avg_quantity = quantity.mean()
avg_unit_price = unit_price.mean()

# Clientes únicos que aparecen al menos una vez en transacciones
unique_customers_in_tx = np.unique(tx_customer_id).size

print("Resumen (transacciones)")
print("-" * 40)
print(f"Transacciones (conteo): {count_transactions}")
print(f"Clientes distintos con compras: {unique_customers_in_tx}")
print(f"Ventas totales (suma): {total_sales:.0f}")
print(f"Ticket promedio (media): {avg_ticket:.0f}")
print(f"Ticket máximo: {max_ticket:.0f}")
print(f"Ticket mínimo: {min_ticket:.0f}")
print(f"Cantidad promedio: {avg_quantity:.2f}")
print(f"Precio unitario promedio: {avg_unit_price:.0f}")

# -------------------------
# 5) Guardado de datos (insumo para Pandas en la Lección 2)
# -------------------------
np.save("customers.npy", customers)
np.save("transactions.npy", transactions)

print("\nArchivos guardados: customers.npy, transactions.npy")


Resumen (transacciones)
----------------------------------------
Transacciones (conteo): 5000
Clientes distintos con compras: 500
Ventas totales (suma): 656650416
Ticket promedio (media): 131330
Ticket máximo: 1067112
Ticket mínimo: 753
Cantidad promedio: 6.55
Precio unitario promedio: 20056

Archivos guardados: customers.npy, transactions.npy


## step 2 — Exploración de datos con Pandas
Carga de datos `.npy`, conversión a DataFrame y exploración inicial.

In [6]:
from IPython.display import display
import pandas as pd

# Mostrar floats sin notación científica (solo visual)
pd.options.display.float_format = '{:,.0f}'.format

# ============================================
# Lección 2 — Pandas
# Conversión a DataFrames, exploración inicial
# integración en 1 solo DataFrame
# ============================================

# 1) Convertir arrays NumPy a DataFrames
customers_df = pd.DataFrame(
    customers,
    columns=["customer_id", "age", "tenure_months"]
)

transactions_df = pd.DataFrame(
    transactions,
    columns=["transaction_id", "customer_id", "quantity", "unit_price", "total"]
)

# 2) Integrar (merge) clientes + transacciones usando customer_id
# Cada transacción queda enriquecida con datos del cliente (edad, antigüedad)
df_integrado = transactions_df.merge(
    customers_df,
    on="customer_id",
    how="left"
)

# --------------------------------------------
# Exploración inicial (sobre el DataFrame integrado)
# --------------------------------------------
display(df_integrado.head())
display(df_integrado.tail())
display(df_integrado.describe())

# --------------------------------------------
# Filtros condicionales (sobre el integrado)
# --------------------------------------------

# Transacciones de clientes mayores de 50
tx_clientes_mayores_50 = df_integrado[df_integrado["age"] > 50]

# Transacciones con total > 100.000
tx_mayor_100k = df_integrado[df_integrado["total"] > 100000]

display(tx_clientes_mayores_50.head())
display(tx_mayor_100k.head())

# --------------------------------------------
# Guardado en Uun solo CSV 
# --------------------------------------------
df_integrado.to_csv("dataset_preliminar_integrado.csv", index=False)

print("Archivo CSV guardado: dataset_preliminar_integrado.csv")


,transaction_id,customer_id,quantity,unit_price,total,age,tenure_months
0,1,10339,9,48947,440523,53,25
1,2,10471,6,10156,60936,61,37
2,3,10068,11,4441,48851,53,44
3,4,10033,9,11646,104814,44,4
4,5,10261,7,13557,94899,33,9


,transaction_id,customer_id,quantity,unit_price,total,age,tenure_months
4995,4996,10263,6,24179,145074,26,18
4996,4997,10178,8,28116,224928,24,6
4997,4998,10493,2,20478,40956,43,28
4998,4999,10489,4,3439,13756,36,41
4999,5000,10208,11,42644,469084,26,57


,transaction_id,customer_id,quantity,unit_price,total,age,tenure_months
count,"5,000","5,000","5,000","5,000","5,000","5,000","5,000"
mean,"2,500","10,249",7,"20,056","131,330",44,31
std,"1,444",146,3,"14,166","125,295",15,18
min,1,"10,000",1,500,753,18,0
25%,"1,251","10,123",4,"9,805","42,246",32,16
50%,"2,500","10,251",7,"16,694","92,942",44,31
75%,"3,750","10,377",10,"27,134","177,320",57,47
max,"5,000","10,499",12,"116,349","1,067,112",70,60


,transaction_id,customer_id,quantity,unit_price,total,age,tenure_months
0,1,10339,9,48947,440523,53,25
1,2,10471,6,10156,60936,61,37
2,3,10068,11,4441,48851,53,44
7,8,10204,12,16483,197796,56,57
23,24,10007,8,16526,132208,56,31


,transaction_id,customer_id,quantity,unit_price,total,age,tenure_months
0,1,10339,9,48947,440523,53,25
3,4,10033,9,11646,104814,44,4
6,7,10449,12,13376,160512,32,53
7,8,10204,12,16483,197796,56,57
9,10,10234,6,17961,107766,19,1


Archivo CSV guardado: dataset_preliminar_integrado.csv


### Hallazgos y utilidad de Pandas

Durante la exploración inicial de los datos generados con NumPy, la utilización de Pandas permitió transformar arrays numéricos en estructuras tabulares (DataFrames) mucho más fáciles de analizar y manipular. A través de funciones como `head()`, fue posible visualizar rápidamente los primeros registros y verificar la correcta estructura de los datos.

Pandas facilita la exploración de datasets mediante métodos integrados para inspeccionar información, aplicar filtros y preparar los datos para análisis posteriores. Además, su capacidad para exportar datos a formatos como CSV simplifica la integración con otras etapas del flujo de trabajo, permitiendo reutilizar los datos en procesos de limpieza, transformación y análisis más avanzados.


## step 3 — Obtención de datos desde archivos
Lectura de archivos CSV, Excel y tablas web y consolidacion.

In [7]:
# ============================================
# Lección 3 — Integración de múltiples fuentes
# CSV integrado + Excel + HTML (read_html)
# ============================================

import pandas as pd
from IPython.display import display

# --------------------------------------------
# 1) Cargar archivo CSV integrado (Lección 2)
# --------------------------------------------

df_base = pd.read_csv("dataset_preliminar_integrado.csv")

display(df_base.head())

# --------------------------------------------
# 2) Incorporar nuevas fuentes de datos
# --------------------------------------------

# 2.1 Leer archivo Excel (país por cliente)
customers_country_df = pd.read_excel("customers_country_latam.xlsx")
display(customers_country_df.head())

# 2.2 Leer archivo HTML local (método de pago por cliente)
customer_payment_df = pd.read_html("customer_payment_method.html")[0]
display(customer_payment_df.head())

# --------------------------------------------
# 3) Unificar todas las fuentes en un único DataFrame
# --------------------------------------------

# Agregar país al dataset base
df_enriched = df_base.merge(
    customers_country_df,
    on="customer_id",
    how="left"
)

# Agregar método de pago al dataset
df_enriched = df_enriched.merge(
    customer_payment_df,
    on="customer_id",
    how="left"
)

display(df_enriched.head())

# --------------------------------------------
# 4) Guardar DataFrame consolidado final
# --------------------------------------------

df_enriched.to_csv("dataset_consolidado.csv", index=False)

print("Archivo generado: dataset_consolidado.csv")


,transaction_id,customer_id,quantity,unit_price,total,age,tenure_months
0,1,10339,9,48947,440523,53,25
1,2,10471,6,10156,60936,61,37
2,3,10068,11,4441,48851,53,44
3,4,10033,9,11646,104814,44,4
4,5,10261,7,13557,94899,33,9


,customer_id,country
0,10000,Uruguay
1,10001,Colombia
2,10002,Paraguay
3,10003,México
4,10004,Uruguay


,customer_id,payment_method
0,10000,Efectivo
1,10001,PayPal
2,10002,Transferencia
3,10003,PayPal
4,10004,PayPal


,transaction_id,customer_id,quantity,unit_price,total,age,tenure_months,country,payment_method
0,1,10339,9,48947,440523,53,25,Ecuador,Efectivo
1,2,10471,6,10156,60936,61,37,Colombia,PayPal
2,3,10068,11,4441,48851,53,44,Uruguay,Efectivo
3,4,10033,9,11646,104814,44,4,Perú,CrÃ©dito
4,5,10261,7,13557,94899,33,9,Argentina,Transferencia


Archivo generado: dataset_consolidado.csv


## step 4 — Manejo de valores perdidos y outliers
Aplicar técnicas de limpieza de datos, resolviendo
problemas de valores nulos y datos atípicos.

In [8]:
# ============================================
# Lección 4 — Limpieza de datos (Pandas)
# Valores nulos + outliers (IQR)
# Input : dataset_consolidado.csv (step 3)
# Output: dataset_limpio.csv
# ============================================

import pandas as pd
from IPython.display import display

# 1) Cargar dataset consolidado
df = pd.read_csv("dataset_consolidado.csv")
print("Shape inicial:", df.shape)
display(df.head())

# 2) Detectar valores nulos
print("\nNulos por columna (antes):")
display(df.isnull().sum())

# 3) Tratar valores nulos
# - Categóricas: rellenar con "Desconocido"
# - Numéricas: rellenar con mediana
categorical_cols = [c for c in ["country", "payment_method"] if c in df.columns]
numeric_cols = [c for c in ["age", "tenure_months", "quantity", "unit_price", "total"] if c in df.columns]

for c in categorical_cols:
    df[c] = df[c].fillna("Desconocido")

for c in numeric_cols:
    df[c] = df[c].fillna(df[c].median())

print("\nNulos por columna (después):")
display(df.isnull().sum())

# 4) Detectar y tratar outliers (IQR) en columnas numéricas clave
# Estrategia: capado (winsorización) con clip usando límites IQR
def cap_outliers_iqr(dataframe, col, k=1.5):
    q1 = dataframe[col].quantile(0.25)
    q3 = dataframe[col].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - k * iqr
    upper = q3 + k * iqr

    outliers_mask = (dataframe[col] < lower) | (dataframe[col] > upper)
    outliers_count = int(outliers_mask.sum())

    dataframe[col] = dataframe[col].clip(lower, upper)
    return lower, upper, outliers_count

outlier_cols = [c for c in ["total", "unit_price", "quantity"] if c in df.columns]

print("\nOutliers detectados y capados (IQR):")
for c in outlier_cols:
    lower, upper, n_out = cap_outliers_iqr(df, c, k=1.5)
    print(f"- {c}: {n_out} outliers | límites [{lower:.2f}, {upper:.2f}]")

# 5) Verificación rápida post-limpieza
print("\nResumen estadístico (post-limpieza):")
display(df[numeric_cols].describe())

# 6) Guardar dataset limpio
df.to_csv("dataset_limpio.csv", index=False)
print("\nArchivo generado: dataset_limpio.csv")
print("Shape final:", df.shape)


Shape inicial: (5000, 9)


,transaction_id,customer_id,quantity,unit_price,total,age,tenure_months,country,payment_method
0,1,10339,9,48947,440523,53,25,Ecuador,Efectivo
1,2,10471,6,10156,60936,61,37,Colombia,PayPal
2,3,10068,11,4441,48851,53,44,Uruguay,Efectivo
3,4,10033,9,11646,104814,44,4,Perú,CrÃ©dito
4,5,10261,7,13557,94899,33,9,Argentina,Transferencia



Nulos por columna (antes):


transaction_id    0
customer_id       0
quantity          0
unit_price        0
total             0
age               0
tenure_months     0
country           0
payment_method    0
dtype: int64


Nulos por columna (después):


transaction_id    0
customer_id       0
quantity          0
unit_price        0
total             0
age               0
tenure_months     0
country           0
payment_method    0
dtype: int64


Outliers detectados y capados (IQR):
- total: 268 outliers | límites [-160364.25, 379929.75]
- unit_price: 163 outliers | límites [-16189.88, 53129.12]
- quantity: 0 outliers | límites [-5.00, 19.00]

Resumen estadístico (post-limpieza):


,age,tenure_months,quantity,unit_price,total
count,"5,000","5,000","5,000","5,000","5,000"
mean,44,31,7,"19,690","125,036"
std,15,18,3,"12,986","105,443"
min,18,0,1,500,753
25%,32,16,4,"9,805","42,246"
50%,44,31,7,"16,694","92,942"
75%,57,47,10,"27,134","177,320"
max,70,60,12,"53,129","379,930"



Archivo generado: dataset_limpio.csv
Shape final: (5000, 9)


In [9]:
# Ejemplo: eliminar duplicados
transactions_df = transactions_df.drop_duplicates()


## step 5 — DATA WRANGLING
Transformar y enriquecer los datos mediante técnicas de
manipulación avanzada.

In [10]:
# ============================================
# Lección 5 — Data Wrangling
# Input : dataset_limpio.csv (Lección 4)
# Output: dataset_transformado.csv
# ============================================

import pandas as pd
from IPython.display import display

# 1) Tomar el DataFrame limpio
df = pd.read_csv("dataset_limpio.csv")
print("Shape inicial:", df.shape)

# --------------------------------------------
# 2) Data Wrangling
# --------------------------------------------

# 2.1 Eliminar duplicados (por seguridad)
# En este caso, una transacción debería ser única por transaction_id
df = df.drop_duplicates(subset=["transaction_id"])
print("Shape después de eliminar duplicados:", df.shape)

# 2.2 Transformar tipos de datos
# Asegurar enteros en columnas que no deberían tener decimales
int_cols = ["transaction_id", "customer_id", "quantity", "unit_price", "total", "age", "tenure_months"]
int_cols = [c for c in int_cols if c in df.columns]
for c in int_cols:
    df[c] = df[c].astype(int)

# Convertir categorías (ahorra memoria y mejora análisis)
cat_cols = ["country", "payment_method"]
cat_cols = [c for c in cat_cols if c in df.columns]
for c in cat_cols:
    df[c] = df[c].astype("category")

# 2.3 Crear nuevas columnas calculadas
df["avg_price_per_unit"] = df["total"] / df["quantity"]

# Flag: transacción de alto valor (lambda)
df["high_value_tx"] = df["total"].apply(lambda x: x > 150000)

# 2.4 Funciones personalizadas (apply/map/lambda)
# map: normalizar nombres de método de pago (arregla variantes típicas)
payment_map = {
    "CrÃ©dito": "Crédito",
    "Credito": "Crédito",
    "Débito": "Débito",
    "Debito": "Débito"
}
if "payment_method" in df.columns:
    df["payment_method"] = df["payment_method"].astype(str).map(lambda x: payment_map.get(x, x)).astype("category")

# apply: etiqueta de ticket (bajo/medio/alto) según total
def ticket_label(x):
    if x < 50000:
        return "Bajo"
    elif x < 150000:
        return "Medio"
    else:
        return "Alto"

df["ticket_label"] = df["total"].apply(ticket_label).astype("category")

# 2.5 Discretizar / normalizar
# Discretizar edad y antigüedad
df["age_group"] = pd.cut(
    df["age"],
    bins=[17, 30, 45, 60, 100],
    labels=["18-30", "31-45", "46-60", "60+"]
)

df["tenure_group"] = pd.cut(
    df["tenure_months"],
    bins=[-1, 12, 36, 60],
    labels=["0-12 meses", "13-36 meses", "37-60 meses"]
)

# (Opcional) Normalización simple (min-max) del total, útil para modelos
df["total_norm"] = (df["total"] - df["total"].min()) / (df["total"].max() - df["total"].min())

# --------------------------------------------
# 3) Guardar DataFrame optimizado
# --------------------------------------------
display(df.head())
df.to_csv("dataset_transformado.csv", index=False)

print("Archivo generado: dataset_transformado.csv")
print("Shape final:", df.shape)


Shape inicial: (5000, 9)
Shape después de eliminar duplicados: (5000, 9)


,transaction_id,customer_id,quantity,unit_price,total,age,tenure_months,country,payment_method,avg_price_per_unit,high_value_tx,ticket_label,age_group,tenure_group,total_norm
0,1,10339,9,48947,379929,53,25,Ecuador,Efectivo,"42,214",True,Alto,46-60,13-36 meses,1
1,2,10471,6,10156,60936,61,37,Colombia,PayPal,"10,156",False,Medio,60+,37-60 meses,0
2,3,10068,11,4441,48851,53,44,Uruguay,Efectivo,"4,441",False,Bajo,46-60,37-60 meses,0
3,4,10033,9,11646,104814,44,4,Perú,Crédito,"11,646",False,Medio,31-45,0-12 meses,0
4,5,10261,7,13557,94899,33,9,Argentina,Transferencia,"13,557",False,Medio,31-45,0-12 meses,0


Archivo generado: dataset_transformado.csv
Shape final: (5000, 15)


## step 6 —  Agrupamiento y pivoteo de datos
 Organizar y estructurar los datos para el análisis
utilizando técnicas de agrupamiento y pivotado

In [11]:
# ============================================
# Lección 6 — Agrupamiento, pivot/melt y exportación final
# Input : dataset_transformado.csv (Lección 5)
# Outputs: dataset_final.csv, dataset_final.xlsx
# ============================================

import pandas as pd
from IPython.display import display

df = pd.read_csv("dataset_transformado.csv")

# groupby: métricas resumidas por país
resumen_pais = (
    df.groupby("country")
      .agg(
          ventas_totales=("total", "sum"),
          ticket_promedio=("total", "mean"),
          transacciones=("transaction_id", "count")
      )
      .reset_index()
)

display(resumen_pais.head())

# pivot: matriz país x método de pago con suma de ventas
pivot_pais_pago = df.pivot_table(
    index="country",
    columns="payment_method",
    values="total",
    aggfunc="sum",
    fill_value=0
)

display(pivot_pais_pago.head())

# melt: volver pivot a formato “largo” (útil para gráficos)
melt_pais_pago = pivot_pais_pago.reset_index().melt(
    id_vars="country",
    var_name="payment_method",
    value_name="ventas_totales"
)

display(melt_pais_pago.head())

# merge: ejemplo uniendo resumen_pais con el promedio de edad por país
edad_prom_pais = df.groupby("country")["age"].mean().reset_index(name="edad_promedio")
resumen_pais = resumen_pais.merge(edad_prom_pais, on="country", how="left")

# concat: ejemplo combinando tablas resumen (simple y demostrativo)
tablas_resumen = pd.concat(
    [resumen_pais.assign(tabla="resumen_pais"),
     melt_pais_pago.assign(tabla="ventas_por_pago")],
    ignore_index=True,
    sort=False
)

# Exportación final
df.to_csv("dataset_final.csv", index=False)
df.to_excel("dataset_final.xlsx", index=False)

print("Archivos generados: dataset_final.csv, dataset_final.xlsx")


,country,ventas_totales,ticket_promedio,transacciones
0,Argentina,55564511,"122,930",452
1,Bolivia,57537226,"131,966",436
2,Brasil,43507830,"116,331",374
3,Chile,75395636,"124,415",606
4,Colombia,58218840,"124,933",466


payment_method,Crédito,DÃ©bito,Efectivo,PayPal,Transferencia
country,,,,,
Argentina,8007637,14307833,14759934,10795616,7693491
Bolivia,13148488,9187350,21216647,5886925,8097816
Brasil,10492093,9054329,9386627,5751810,8822971
Chile,17401577,10429402,18921615,16728693,11914349
Colombia,12712319,17177517,14799181,7052493,6477330


,country,payment_method,ventas_totales
0,Argentina,Crédito,8007637
1,Bolivia,Crédito,13148488
2,Brasil,Crédito,10492093
3,Chile,Crédito,17401577
4,Colombia,Crédito,12712319


Archivos generados: dataset_final.csv, dataset_final.xlsx


🧩 Resumen del flujo de trabajo del proyecto

El proyecto se desarrolló siguiendo un flujo completo de preparación y análisis de datos, dividido en seis etapas consecutivas, utilizando principalmente NumPy y Pandas.

🔹 Lección 1 — Generación de datos con NumPy

Se crearon datasets ficticios de clientes y transacciones utilizando arrays de NumPy.
En esta etapa se aplicaron operaciones matemáticas básicas como suma, promedio, máximos, mínimos y conteos, y se guardaron los datos como archivos .npy.
El objetivo fue simular datos reales y comprender el uso eficiente de NumPy para el manejo de información numérica.

🔹 Lección 2 — Exploración inicial con Pandas

Los arrays generados fueron convertidos en DataFrames de Pandas.
Se realizó una exploración inicial mediante visualización de primeras y últimas filas, estadísticas descriptivas y filtros condicionales.
Finalmente, se exportó un dataset preliminar en formato CSV, que sirvió como base para las siguientes etapas.

🔹 Lección 3 — Integración de múltiples fuentes de datos

Se incorporaron fuentes de datos complementarias desde distintos formatos:

CSV (dataset base),

Excel (información adicional por cliente),

HTML local utilizando read_html().

Estas fuentes se unificaron mediante operaciones de merge, obteniendo un dataset consolidado que combina información transaccional y descriptiva.

🔹 Lección 4 — Limpieza de datos

Se identificaron y trataron valores nulos, aplicando estrategias distintas según el tipo de dato.
Además, se detectaron outliers en variables numéricas utilizando el método IQR, y se aplicó capado (winsorización) para mantener la integridad del dataset sin eliminar registros.
El resultado fue un dataset limpio, listo para ser transformado.

🔹 Lección 5 — Data Wrangling

Se aplicaron técnicas de transformación y enriquecimiento de datos, incluyendo:

eliminación de registros duplicados,

conversión de tipos de datos,

creación de nuevas columnas calculadas,

uso de funciones apply(), map() y lambda,

discretización de variables numéricas en rangos.

Esta etapa permitió optimizar el dataset y prepararlo para análisis más avanzados.

🔹 Lección 6 — Agregación y reestructuración

Se generaron métricas resumidas mediante groupby().
Los datos fueron reestructurados usando pivot_table() y melt(), y se realizaron combinaciones adicionales con merge() y concat() cuando fue necesario.
Finalmente, se exportó el dataset final listo para análisis en formatos CSV y Excel.